# S3 데이터셋 적재를 위한 테스트 데이터 필터링
- 기간: 220601 ~ 220607

In [1]:
import numpy as np
import pandas as pd
import os
import shutil
import copy
from tqdm import tqdm
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings(action='ignore')
pd.set_option("display.max_columns", None)

## 1. 데이터셋 필터링

In [2]:
# 현재 작업 디렉토리 가져오기
original_directory = os.getcwd()
print(f"원래 디렉토리: {original_directory}")

원래 디렉토리: c:\Users\aryij\Documents\DataStudy\epoch\final-project


In [3]:
# # poi 데이터는 날짜 컬럼 없음
# # poi_df = pd.read_csv(os.path.join("data","aihub", "metadata", "tn_poi_master_POIMaster_all.csv"))
# poi_df = pd.read_parquet(os.path.join("data","aihub", "metadata", "tn_poi_master_POIMaster_all.parquet"))
# poi_df.head(3)

In [4]:
path_parquet_22 = os.path.join("data", "aihub", "2022", "total_combined", "gps_data_parquet", "total_gps_2022.parquet")
path_parquet_23 = os.path.join("data", "aihub", "2023", "total_combined", "gps_data_parquet", "total_gps_2023.parquet")
gps_22 = pd.read_parquet(path_parquet_22, engine="pyarrow")
gps_23 = pd.read_parquet(path_parquet_23, engine="pyarrow")

In [5]:
gps_22.head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
0,a9c1a065c89da3b7,126.702718,37.500779,a_a000011,capital,2022-08-06,12:33:00
1,a9c1a065c89da3b7,126.703540,37.506400,a_a000011,capital,2022-08-06,12:34:00
2,a9c1a065c89da3b7,126.706564,37.506687,a_a000011,capital,2022-08-06,12:35:00


In [6]:
gps_23.head(3)

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
0,e000004_900009,127.054289,37.209919,e_e000004,capital,2023-04-30,13:30:00
1,e000004_900009,127.053911,37.208667,e_e000004,capital,2023-04-30,13:30:00
2,e000004_900009,127.053411,37.207996,e_e000004,capital,2023-04-30,13:31:00


In [7]:
# gps 데이터 언제부터 있는지 확인
print(f'2022년 가장 빠른 일자: {np.min(gps_22["DT_YMD"])}')
print(f'2022년 가장 늦은 일자: {np.max(gps_22["DT_YMD"])}')

2022년 가장 빠른 일자: 2022-08-03
2022년 가장 늦은 일자: 2022-11-14


In [8]:
print(f'2023년 가장 빠른 일자: {np.min(gps_23["DT_YMD"])}')
print(f'2023년 가장 늦은 일자: {np.max(gps_23["DT_YMD"])}')

2023년 가장 빠른 일자: 2023-04-28
2023년 가장 늦은 일자: 2023-09-30


In [9]:
region_22 = os.listdir(os.path.join("data", "aihub", "2022", "total_combined", "region_data"))
region_23 = os.listdir(os.path.join("data", "aihub", "2023", "total_combined", "region_data"))

In [10]:
def check_date(directory):
    """
    특정 디렉터리 내 CSV 파일에서 "YMD"라는 글자를 포함한 컬럼의 최소값과 최대값을 출력하는 함수.

    Parameters:
        directory (str): 검색할 디렉터리 경로.
    """
    # 디렉터리 내용 가져오기
    all_files = os.listdir(directory)
    
    # .csv 파일 필터링
    csv_files = [f for f in all_files if f.endswith('.csv') and os.path.isfile(os.path.join(directory, f))]
    
    for csv_file in csv_files:
        # CSV 파일 읽기
        df = pd.read_csv(os.path.join(directory, csv_file))

        # "YMD"라는 글자를 포함한 컬럼 찾기
        if csv_file == 'tn_traveller_master_여행객 Master':
            ymd_columns = ["TRAVEL_STATUS_START_YMD", "TRAVEL_STATUS_END_YMD"]
        else:
            ymd_columns = [col for col in df.columns if "YMD" in col]

        for col in ymd_columns:
            # "YMD" 컬럼에서 최소값 및 최대값 찾기 (errors='coerce'로 에러 방지)
            dates = pd.to_datetime(df[col], errors='coerce')

            # 유효한 날짜만 남기기
            valid_dates = dates.dropna()

            if not valid_dates.empty:
                min_date = valid_dates.min()
                max_date = valid_dates.max()

                print(f'{csv_file}, {col} 최소값: {min_date}')
                print(f'{csv_file}, {col} 최대값: {max_date}\n')
            else:
                print(f'{csv_file}, {col}에는 유효한 날짜가 없습니다.\n')


In [11]:
check_date(os.path.join("data", "aihub", "2022", "total_combined", "region_data"))

tn_activity_consume_his_활동소비내역.csv, PAYMENT_DT_YMD 최소값: 2022-06-01 00:00:00
tn_activity_consume_his_활동소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-02-28 00:00:00

tn_adv_consume_his_사전소비내역.csv, PAYMENT_DT_YMD 최소값: 2022-01-01 00:00:00
tn_adv_consume_his_사전소비내역.csv, PAYMENT_DT_YMD 최대값: 2022-11-25 00:00:00

tn_lodge_consume_his_숙박소비내역.csv, CHK_YMD 최소값: 2022-08-03 00:00:00
tn_lodge_consume_his_숙박소비내역.csv, CHK_YMD 최대값: 2022-11-13 00:00:00

tn_lodge_consume_his_숙박소비내역.csv, CHK_OUT_DT_YMD 최소값: 2022-08-04 00:00:00
tn_lodge_consume_his_숙박소비내역.csv, CHK_OUT_DT_YMD 최대값: 2022-11-14 00:00:00

tn_lodge_consume_his_숙박소비내역.csv, PAYMENT_DT_YMD 최소값: 2022-01-01 00:00:00
tn_lodge_consume_his_숙박소비내역.csv, PAYMENT_DT_YMD 최대값: 2022-11-12 00:00:00

tn_move_his_이동내역.csv, START_DT_YMD 최소값: 2022-08-03 00:00:00
tn_move_his_이동내역.csv, START_DT_YMD 최대값: 2022-11-13 00:00:00

tn_move_his_이동내역.csv, END_DT_YMD 최소값: 2022-08-03 00:00:00
tn_move_his_이동내역.csv, END_DT_YMD 최대값: 2022-11-13 00:00:00

tn_mvmn_consume_his_이동수단소비내역.csv, PAYME

In [12]:
check_date(os.path.join("data", "aihub", "2023", "total_combined", "region_data"))

tn_activity_consume_his_활동소비내역.csv, PAYMENT_DT_YMD 최소값: 2023-04-04 00:00:00
tn_activity_consume_his_활동소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-09-30 00:00:00

tn_adv_consume_his_사전소비내역.csv, PAYMENT_DT_YMD 최소값: 2023-03-14 00:00:00
tn_adv_consume_his_사전소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-09-28 00:00:00

tn_lodge_consume_his_숙박소비내역.csv, CHK_YMD에는 유효한 날짜가 없습니다.

tn_lodge_consume_his_숙박소비내역.csv, CHK_OUT_DT_YMD에는 유효한 날짜가 없습니다.

tn_lodge_consume_his_숙박소비내역.csv, PAYMENT_DT_YMD 최소값: 2023-01-01 00:00:00
tn_lodge_consume_his_숙박소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-09-17 00:00:00

tn_move_his_이동내역.csv, START_DT_YMD 최소값: 2023-04-28 00:00:00
tn_move_his_이동내역.csv, START_DT_YMD 최대값: 2023-09-30 00:00:00

tn_move_his_이동내역.csv, END_DT_YMD 최소값: 2023-04-28 00:00:00
tn_move_his_이동내역.csv, END_DT_YMD 최대값: 2023-09-30 00:00:00

tn_mvmn_consume_his_이동수단소비내역.csv, PAYMENT_DT_YMD 최소값: 2023-01-03 00:00:00
tn_mvmn_consume_his_이동수단소비내역.csv, PAYMENT_DT_YMD 최대값: 2023-09-28 00:00:00

tn_tour_photo_관광사진.csv, PHOTO_FILE_DT_YMD 최소값: 

In [13]:
def filter_by_date_region(dir1="aihub", dir2="2022", start_date="2022-01-02", end_date="2022-01-08"):
    
    # 기본 경로 설정 (data\aihub\2022)
    if start_date.startswith("2022"):       
        base_path = os.path.join("data", dir1, dir2, "total_combined", "region_data")
    else:
        dir2 = "2023"
        base_path = os.path.join("data", dir1, dir2, "total_combined", "region_data")

    dataset_list = os.listdir(base_path)

    region_dfs = {}

    # gps_data, region_data 폴더 내부를 for loop으로 들어간다
    for dataset in dataset_list:
        dataset_path = os.path.join(base_path, dataset)

        if not dataset.endswith('.csv'):
            print(f"파일 스킵: {dataset} (CSV 파일 아님)")
            continue

        try:
            # 파일 읽기 시도
            df = pd.read_csv(dataset_path)
            df_name = dataset.replace(".csv", "")
            
            # 파일에 데이터가 없는 경우 스킵
            if df.empty:
                print(f"파일 스킵: {dataset} (데이터 없음)")
                continue

            # 데이터프레임에서 "YMD"로 끝나는 컬럼 찾는다 (START_DATE_YMD는 제외)
            ymd_columns = [col for col in df.columns if col.endswith('YMD') and "START" not in col]

            if not ymd_columns:
                print(f"YMD 컬럼 없음: {dataset}")

                region_dfs[df_name] = df
                
                continue
            
            if len(ymd_columns) > 1:
                ymd_columns = [ymd_columns[-1]]

            # print(f"데이터셋: {dataset}\nYMD 컬럼: {ymd_columns}\n")

            # YMD 컬럼 처리
            for ymd_col in ymd_columns:
                df[ymd_col] = pd.to_datetime(df[ymd_col], errors="coerce")

            # 날짜 필터링 (ymd_columns 중 첫 번째 컬럼)
            filtered_df = df[(df[ymd_columns[0]] >= start_date) & (df[ymd_columns[0]] <= end_date)]

            region_dfs[df_name] = filtered_df

        except pd.errors.EmptyDataError:
            print(f"파일 스킵: {dataset} (비어 있는 파일)")
        except Exception as e:
            print(f"파일 처리 오류: {dataset}: {e}")

    print(f"===================== {dir2} 데이터 필터링 완료 ({start_date} ~ {end_date}) =====================\n")

    return region_dfs

In [14]:
# gps 데이터 필터링 함수
def filter_by_date_gps(df, ymd_col="DT_YMD", start_date="2022-01-02", end_date="2022-01-08"):

    df[ymd_col] = pd.to_datetime(df[ymd_col], errors="coerce")
    filtered_df = df[(df[ymd_col] >= start_date) & (df[ymd_col] <= end_date)]

    print(f"===================== gps 데이터 필터링 완료 ({start_date} ~ {end_date}) =====================\n")

    return filtered_df

## 2. 날짜 기준 초기 값 세팅
- 2022-01-02 ~ 2023-06-03까지 데이터를 초기 값으로 넣는다
- S3에 한 번만 넣어주면 된다

In [15]:
# 2022-01-02 ~ 2023-06-03까지 일주일 단위로 딕셔너리 생성
# 일주일 단위의 시작 날짜와 끝 날짜를 포함한 데이터 (이중 딕셔너리)

weekly_default_dict = {
    f"week_{i+1}": {
        "start_date": str(start.date()),
        "end_date": str((start + pd.Timedelta(days=6)).date())
    }
    for i, start in enumerate(pd.date_range(start="2022-01-02", end="2023-06-03", freq="7D"))
}
weekly_default_dict

{'week_1': {'start_date': '2022-01-02', 'end_date': '2022-01-08'},
 'week_2': {'start_date': '2022-01-09', 'end_date': '2022-01-15'},
 'week_3': {'start_date': '2022-01-16', 'end_date': '2022-01-22'},
 'week_4': {'start_date': '2022-01-23', 'end_date': '2022-01-29'},
 'week_5': {'start_date': '2022-01-30', 'end_date': '2022-02-05'},
 'week_6': {'start_date': '2022-02-06', 'end_date': '2022-02-12'},
 'week_7': {'start_date': '2022-02-13', 'end_date': '2022-02-19'},
 'week_8': {'start_date': '2022-02-20', 'end_date': '2022-02-26'},
 'week_9': {'start_date': '2022-02-27', 'end_date': '2022-03-05'},
 'week_10': {'start_date': '2022-03-06', 'end_date': '2022-03-12'},
 'week_11': {'start_date': '2022-03-13', 'end_date': '2022-03-19'},
 'week_12': {'start_date': '2022-03-20', 'end_date': '2022-03-26'},
 'week_13': {'start_date': '2022-03-27', 'end_date': '2022-04-02'},
 'week_14': {'start_date': '2022-04-03', 'end_date': '2022-04-09'},
 'week_15': {'start_date': '2022-04-10', 'end_date': '202

### 2-1. Region 데이터

In [16]:
def get_weekly_region(weekly_dict):
    region_dict = {}
    
    for key in weekly_dict:

        start_date = weekly_dict[key]["start_date"]
        end_date = weekly_dict[key]["end_date"]

        region_dict[start_date] = filter_by_date_region(dir1="aihub", dir2="2022", 
                                                        start_date=start_date, end_date=end_date)
    
    return region_dict

In [17]:
region_dict_default = get_weekly_region(weekly_default_dict)

YMD 컬럼 없음: tc_codea_코드A.csv
YMD 컬럼 없음: tc_codeb_코드B.csv
YMD 컬럼 없음: tc_sgg_시군구코드.csv
YMD 컬럼 없음: tn_activity_his_활동내역.csv
YMD 컬럼 없음: tn_companion_info_동반자정보.csv
===================== 2022 데이터 필터링 완료 (2022-01-02 ~ 2022-01-08) =====================

YMD 컬럼 없음: tc_codea_코드A.csv
YMD 컬럼 없음: tc_codeb_코드B.csv
YMD 컬럼 없음: tc_sgg_시군구코드.csv
YMD 컬럼 없음: tn_activity_his_활동내역.csv
YMD 컬럼 없음: tn_companion_info_동반자정보.csv
===================== 2022 데이터 필터링 완료 (2022-01-09 ~ 2022-01-15) =====================

YMD 컬럼 없음: tc_codea_코드A.csv
YMD 컬럼 없음: tc_codeb_코드B.csv
YMD 컬럼 없음: tc_sgg_시군구코드.csv
YMD 컬럼 없음: tn_activity_his_활동내역.csv
YMD 컬럼 없음: tn_companion_info_동반자정보.csv
===================== 2022 데이터 필터링 완료 (2022-01-16 ~ 2022-01-22) =====================

YMD 컬럼 없음: tc_codea_코드A.csv
YMD 컬럼 없음: tc_codeb_코드B.csv
YMD 컬럼 없음: tc_sgg_시군구코드.csv
YMD 컬럼 없음: tn_activity_his_활동내역.csv
YMD 컬럼 없음: tn_companion_info_동반자정보.csv
===================== 2022 데이터 필터링 완료 (2022-01-23 ~ 2022-01-29) =====================

YMD 컬럼 없음: tc_co

In [18]:
# YMD 컬럼 기준으로 csv 파일 필터링
region_dict_default["2023-05-14"]["tn_activity_consume_his_활동소비내역"]

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,ACTIVITY_TYPE_SEQ,CONSUME_HIS_SEQ,CONSUME_HIS_SNO,PAYMENT_NUM,BRNO,STORE_NM,ROAD_NM_ADDR,LOTNO_ADDR,ROAD_NM_CD,LOTNO_CD,PAYMENT_DT,PAYMENT_MTHD_SE,PAYMENT_AMT_WON,PAYMENT_ETC,SGG_CD,REGION,PAYMENT_DT_YMD,PAYMENT_DT_MIN
38,e_e000406,2305210001,3,0,0,2,3,2.208184e+09,주식회사 지마켓,서울 강남구 테헤란로 152,서울 강남구 역삼동 737,3122010.0,1.168010e+09,2023-05-20 00:00:00,1.0,52000.0,동두천 놀자숲 어드벤처,NaN,capital,2023-05-20,00:00:00
52,e_e000396,2305200002,3,0,0,1,2,NaN,NaN,NaN,NaN,NaN,NaN,2023-05-17 00:00:00,3.0,36000.0,레일바이크 이용권,NaN,capital,2023-05-17,00:00:00
57,e_e000362,2305150002,1,0,0,1,2,1.438126e+09,(주)글램비글램핑,경기 화성시 서신면 해안길 64,NaN,4431335.0,4.159035e+09,2023-05-15 15:36:00,1.0,22000.0,숯그릴,NaN,capital,2023-05-15,15:36:00
206,e_e000294,2305190002,1,0,0,1,2,3.078507e+09,정안(상)휴게소,충남 공주시 정안면 논산천안고속도로 58,NaN,1000038.0,4.415037e+09,2023-05-19 13:05:00,1.0,4800.0,알밤크림도넛,NaN,capital,2023-05-19,13:05:00
208,e_e000294,2305190003,1,0,0,1,2,1.018131e+09,한화호텔앤드리조트(주),경기 용인시 처인구 남사읍 봉무로153번길 79,NaN,4409448.0,4.146126e+09,2023-05-19 17:13:00,1.0,84200.0,채끝등심스테이크;비프콜드누들샐러드;트러플치킨크림리조트;라임모히또,NaN,capital,2023-05-19,17:13:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77250,h_h000072,2305190001,2,1,1,1,1,2.692502e+09,귤귤스토어도두점,제주특별자치도 제주시 서해안로 238,제주특별자치도 제주시 도두일동 1733-4,3349093.0,5.011013e+09,2023-05-19 09:58:00,1.0,24100.0,참방향제-귤;참방향제-동백;봉투,NaN,jeju,2023-05-19,09:58:00
77251,h_h000072,2305190006,2,1,1,1,1,6.168215e+09,JDC 면세점,제주특별자치도 제주시 공항로 2,NaN,3349012.0,5.011011e+09,2023-05-19 18:10:00,1.0,31000.0,THIS PLUS,NaN,jeju,2023-05-19,18:10:00
77252,h_h000072,2305170006,6,1,1,1,1,8.835000e+09,로또당구장,제주특별자치도 제주시 연신로 17,제주특별자치도 제주시 일도이동 85-3,3349139.0,5.011010e+09,2023-05-17 21:45:00,1.0,9000.0,NaN,NaN,jeju,2023-05-17,21:45:00
77253,h_h000072,2305180008,1,2,2,1,1,6.168198e+09,주식회사 오슬로,제주특별자치도 제주시 서해안로 442-9,제주특별자치도 제주시 용담삼동 2398,3349093.0,5.011011e+09,2023-05-18 19:25:00,1.0,10000.0,NaN,NaN,jeju,2023-05-18,19:25:00


In [19]:
# YMD 컬럼 (날짜) 없는 csv는 필터링 없이 불러온다
region_dict_default["2023-05-14"]["tn_activity_his_활동내역"]

,TRAVEL_ID,VISIT_AREA_ID,ACTIVITY_TYPE_CD,ACTIVITY_TYPE_SEQ,ACTIVITY_ETC,ACTIVITY_DTL,RSVT_YN,EXPND_SE,ADMISSION_SE,REGION
0,e_e000004,2304300002,3,0,NaN,화성 어차 탑승,Y,5,1.0,capital
1,e_e000004,2304300003,4,0,NaN,NaN,NaN,NaN,NaN,capital
2,e_e000004,2304300004,4,0,NaN,NaN,NaN,NaN,NaN,capital
3,e_e000006,2304300005,1,0,NaN,소금빵,N,1,NaN,capital
4,e_e000006,2304300007,1,0,NaN,쭈꾸미,N,1,NaN,capital
...,...,...,...,...,...,...,...,...,...,...
162070,h_h006731,2309220003,5,1,NaN,NaN,NaN,NaN,NaN,jeju
162071,h_h006753,2309230003,4,0,NaN,NaN,NaN,NaN,NaN,jeju
162072,h_h006753,2309230004,4,0,NaN,NaN,NaN,NaN,NaN,jeju
162073,h_h006753,2309240002,4,0,NaN,NaN,NaN,NaN,NaN,jeju


### 2-2. GPS 데이터

In [20]:
def get_weekly_gps(weekly_dict):

    # gps 파일 불러온다
    path_parquet_22 = os.path.join("data", "aihub", "2022", "total_combined", 
                                   "gps_data_parquet", "total_gps_2022.parquet")
    path_parquet_23 = os.path.join("data", "aihub", "2023", "total_combined", 
                                   "gps_data_parquet", "total_gps_2023.parquet")
    
    gps_22 = pd.read_parquet(path_parquet_22, engine="pyarrow")
    gps_23 = pd.read_parquet(path_parquet_23, engine="pyarrow")

    gps_dict = {}

    # week_53에서 연도 바뀐다 (2023년)
    for key in weekly_dict:
        start_date = weekly_dict[key]["start_date"]
        end_date = weekly_dict[key]["end_date"]

        if start_date.startswith("2022"):
            gps_dict[start_date] = filter_by_date_gps(gps_22, "DT_YMD", start_date, end_date)

        else:
            gps_dict[start_date] = filter_by_date_gps(gps_23, "DT_YMD", start_date, end_date)
    
    return gps_dict

In [21]:
gps_dict_default = get_weekly_gps(weekly_default_dict)

===================== gps 데이터 필터링 완료 (2022-01-02 ~ 2022-01-08) =====================

===================== gps 데이터 필터링 완료 (2022-01-09 ~ 2022-01-15) =====================

===================== gps 데이터 필터링 완료 (2022-01-16 ~ 2022-01-22) =====================

===================== gps 데이터 필터링 완료 (2022-01-23 ~ 2022-01-29) =====================

===================== gps 데이터 필터링 완료 (2022-01-30 ~ 2022-02-05) =====================

===================== gps 데이터 필터링 완료 (2022-02-06 ~ 2022-02-12) =====================

===================== gps 데이터 필터링 완료 (2022-02-13 ~ 2022-02-19) =====================

===================== gps 데이터 필터링 완료 (2022-02-20 ~ 2022-02-26) =====================

===================== gps 데이터 필터링 완료 (2022-02-27 ~ 2022-03-05) =====================

===================== gps 데이터 필터링 완료 (2022-03-06 ~ 2022-03-12) =====================

===================== gps 데이터 필터링 완료 (2022-03-13 ~ 2022-03-19) =====================

===================== gps 데이터 필터링 완료 (2022-03-20 ~ 202

In [22]:
gps_dict_default["2022-10-16"]

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN
1138852,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,12:58:00
1138853,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,12:59:00
1138854,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,13:00:00
1138855,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,13:00:00
1138856,3db6ed280d4982c3,127.740563,38.007263,a_a001698,east,2022-10-17,13:00:00
...,...,...,...,...,...,...,...
62890589,2ED8EF88-813D-4C99-BD29-B716C324CAAA,126.467208,33.503922,d_d007057,jeju,2022-10-22,22:46:00
62890590,2ED8EF88-813D-4C99-BD29-B716C324CAAA,126.467208,33.503922,d_d007057,jeju,2022-10-22,22:46:00
62890591,2ED8EF88-813D-4C99-BD29-B716C324CAAA,126.467620,33.503916,d_d007057,jeju,2022-10-22,22:46:00
62890592,2ED8EF88-813D-4C99-BD29-B716C324CAAA,126.467620,33.503916,d_d007057,jeju,2022-10-22,22:46:00


## 3. 데이터셋 저장

In [23]:
# 지역별 데이터 통합, 저장 함수 생성
def store_filtered_data(data_dict, dir1="aihub", is_gps=True):
    
    # 마지막 코드의 날짜 출력용
    data_keys = list(data_dict.keys())
    first_date = data_keys[0]
    last_date = datetime.strptime(data_keys[-1], "%Y-%m-%d") + timedelta(days=6)
    last_date = last_date.strftime("%Y-%m-%d")

    # 기본 경로 설정 (data\aihub\)
    base_path = os.path.join("data", dir1, "filtered")

    for key, value in data_dict.items():

        start_date = key
        start_plus_6 = datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=6)
        end_date = start_plus_6.strftime("%Y-%m-%d")

        # "2022-01-02" → "220102" 형태로 변경
        start_date_formatted = datetime.strptime(start_date, "%Y-%m-%d").strftime("%y%m%d")
        date_path = os.path.join(base_path, start_date_formatted)

        # "220102" 이름의 폴더 생상
        os.makedirs(date_path, exist_ok=True)

        # data_dict가 gps 데이터일 때
        if is_gps:
            if isinstance(data_dict[key], pd.DataFrame):  # df가 데이터프레임인지 확인
                data_count = 1
            else:
                data_count = 0

            folder_path = os.path.join(date_path, "gps_data")
            os.makedirs(folder_path, exist_ok=True)

            parquet_path = os.path.join(folder_path, f"gps_data_{start_date_formatted}.parquet")
            data_dict[key].to_parquet(parquet_path, engine="pyarrow", index=False)
        
        # data_dict가 region 데이터일 때
        else:
            data_count = len(data_dict[key].keys())

            folder_path = os.path.join(date_path, "region_data")
            os.makedirs(folder_path, exist_ok=True)

            for csv_name in data_dict[key].keys():
            
                csv_path = os.path.join(folder_path, f"{csv_name}_{start_date_formatted}.csv")
                # data_dict[key][csv_name].to_parquet(parquet_path, engine="pyarrow", index=False)
                data_dict[key][csv_name].to_csv(csv_path, index=False)
        
        print(f"------------------ 필터링 데이터 {data_count}개 저장 완료 ({start_date} ~ {end_date}) ------------------")

    print(f"\n=================== 필터링 데이터 총 {len(data_dict.keys())}개 저장 완료 ({first_date} ~ {last_date}) ===================")
    print(f"\n=================== 저장 경로: {folder_path} ===================")


In [24]:
gps_dict_default.keys()

dict_keys(['2022-01-02', '2022-01-09', '2022-01-16', '2022-01-23', '2022-01-30', '2022-02-06', '2022-02-13', '2022-02-20', '2022-02-27', '2022-03-06', '2022-03-13', '2022-03-20', '2022-03-27', '2022-04-03', '2022-04-10', '2022-04-17', '2022-04-24', '2022-05-01', '2022-05-08', '2022-05-15', '2022-05-22', '2022-05-29', '2022-06-05', '2022-06-12', '2022-06-19', '2022-06-26', '2022-07-03', '2022-07-10', '2022-07-17', '2022-07-24', '2022-07-31', '2022-08-07', '2022-08-14', '2022-08-21', '2022-08-28', '2022-09-04', '2022-09-11', '2022-09-18', '2022-09-25', '2022-10-02', '2022-10-09', '2022-10-16', '2022-10-23', '2022-10-30', '2022-11-06', '2022-11-13', '2022-11-20', '2022-11-27', '2022-12-04', '2022-12-11', '2022-12-18', '2022-12-25', '2023-01-01', '2023-01-08', '2023-01-15', '2023-01-22', '2023-01-29', '2023-02-05', '2023-02-12', '2023-02-19', '2023-02-26', '2023-03-05', '2023-03-12', '2023-03-19', '2023-03-26', '2023-04-02', '2023-04-09', '2023-04-16', '2023-04-23', '2023-04-30', '2023-05-

In [25]:
gps_dict_default["2022-01-02"]

,MOBILE_NUM_ID,X_COORD,Y_COORD,TRAVEL_ID,REGION,DT_YMD,DT_MIN


In [26]:
region_dict_default["2022-01-02"].keys()

dict_keys(['tc_codea_코드A', 'tc_codeb_코드B', 'tc_sgg_시군구코드', 'tn_activity_consume_his_활동소비내역', 'tn_activity_his_활동내역', 'tn_adv_consume_his_사전소비내역', 'tn_companion_info_동반자정보', 'tn_lodge_consume_his_숙박소비내역', 'tn_move_his_이동내역', 'tn_mvmn_consume_his_이동수단소비내역', 'tn_tour_photo_관광사진', 'tn_traveller_master_여행객 Master', 'tn_travel_여행', 'tn_visit_area_info_방문지정보'])

In [27]:
store_filtered_data(region_dict_default, dir1="aihub", is_gps=False)

------------------ 필터링 데이터 14개 저장 완료 (2022-01-02 ~ 2022-01-08) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-01-09 ~ 2022-01-15) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-01-16 ~ 2022-01-22) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-01-23 ~ 2022-01-29) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-01-30 ~ 2022-02-05) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-02-06 ~ 2022-02-12) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-02-13 ~ 2022-02-19) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-02-20 ~ 2022-02-26) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-02-27 ~ 2022-03-05) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-03-06 ~ 2022-03-12) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-03-13 ~ 2022-03-19) ------------------
------------------ 필터링 데이터 14개 저장 완료 (2022-03-20 ~ 2022-03-26) ------------------
----------------

In [28]:
store_filtered_data(gps_dict_default, dir1="aihub", is_gps=True)

------------------ 필터링 데이터 1개 저장 완료 (2022-01-02 ~ 2022-01-08) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-01-09 ~ 2022-01-15) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-01-16 ~ 2022-01-22) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-01-23 ~ 2022-01-29) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-01-30 ~ 2022-02-05) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-02-06 ~ 2022-02-12) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-02-13 ~ 2022-02-19) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-02-20 ~ 2022-02-26) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-02-27 ~ 2022-03-05) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-03-06 ~ 2022-03-12) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-03-13 ~ 2022-03-19) ------------------
------------------ 필터링 데이터 1개 저장 완료 (2022-03-20 ~ 2022-03-26) ------------------
------------------ 필터링 데이터 1